In [ ]:
%pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement SerpAPIWrapper (from versions: none)
ERROR: No matching distribution found for SerpAPIWrapper
c:\Users\user\Desktop\llms\genai\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  res = process_handler(cmd, _system_body)
c:\Users\user\Desktop\llms\genai\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  res = process_handler(cmd, _system_body)
c:\Users\user\Desktop\llms\genai\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  res = process_handler(cmd, _system_body)


Built-in Tool - DuckDuckGo Search


In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Top news in Bangladesh.")

print(results)

c:\Users\user\Desktop\llms\genai\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


3 days ago · To know today's all the latest news of The Daily Star, visit our top news page. Get today's news, all news, today's headlines, today's national news, today's international news, … Jun 6, 2025 · Stay updated with the latest BD news, breaking headlines, and live updates from the most reliable bangla news portals in Bangladesh. From politics to entertainment, get … 4 days ago · Bangla Tribune is one of the most revered online newspapers in Bangladesh, due to its reputation of neutral coverage and incisive analysis. 2 days ago · Latest Online English News/Article Covering various niches, including Bangladesh (BD), Sports, Crime, Entertainment, Business, Politics, Education, Opinion, Lifestyle, Photo, … 4 days ago · New Age, a Dhaka-based leading English language daily newspaper, is a major source of news of Bangladesh and international affairs


In [2]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


Built-in Tool - Shell Tool

In [3]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('dir')

print(results)

Executing command:
 dir
 Volume in drive C has no label.
 Volume Serial Number is 624B-89C6

 Directory of c:\Users\user\Desktop\llms\ai_agent

07/08/2025  04:40 PM    <DIR>          .
07/08/2025  04:38 PM    <DIR>          ..
07/09/2025  10:03 AM            11,990 tools-in-langchain.ipynb
               1 File(s)         11,990 bytes
               2 Dir(s)  30,597,488,640 bytes free



c:\Users\user\Desktop\llms\genai\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Custom Tools

In [4]:
from langchain_core.tools import tool

In [5]:
# Step 1 - create a function
def multiply(a,b):
    """Multiply two numbers"""
    return a*b

In [6]:
# Step 2 - add type hints
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a*b

In [7]:
# Step 3 - add tool decorator
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a*b

In [8]:
result = multiply.invoke({'a':3, 'b':5})

In [9]:
print(result)

15


In [10]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


Method 2 - Using StructuredTool

In [12]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [13]:
class MultiplyInput(BaseModel):
    a: int = Field(True, description='The first number to add')
    b: int = Field(True, description = 'The second number to add')

In [14]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [15]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "multiply",
    description= 'Multiply two numbers',
    args_schema=MultiplyInput
)

In [16]:
result = multiply_tool.invoke({'a':3, 'b':3})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'default': True, 'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'default': True, 'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


Method 3 - Using BaseTool Class

In [17]:
from langchain.tools import BaseTool
from typing import Type

In [18]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(True, description="The first number to add")
    b: int = Field(True, description="The second number to add")

In [19]:
class MultiplyTool(BaseTool):
    name: str = "Multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b:int) -> int:
        return a*b


In [20]:
multiply_tool = MultiplyTool()

In [21]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
Multiply
Multiply two numbers
{'a': {'default': True, 'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'default': True, 'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


Toolkit

In [22]:
from langchain_core.tools import tool

In [23]:
# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [24]:
class MathToolkit:
    def get_tools(self):
        return[add, multiply]